In [5]:
from collections import defaultdict

In [6]:
import cis 

ModuleNotFoundError: No module named 'cis'

In [3]:
#segmentation-kitによって得られた各音素の発声タイミングのデータを使い，音声データを切り出し，音素波形を確認します．

# 使用するパッケージ
import matplotlib.pyplot as plt
import cis 
from collections import defaultdict

#音素部分の画像出力
for i in range(1,100,1):
    data_list = []
    open_file = "lab_and_text/sound-"+str(i).zfill(3)+".lab"
    filename = "wav/sound-"+str(i).zfill(3)
    v ,fs = cis.wavread(filename+".wav")
    #labファイルの音素データに関する情報を格納
    with open(open_file,"r") as f:
        data = f.readline().split()
        while data:
            data_list.append(data)
            data = f.readline().split()
        #各音素毎に保存したいので，alphabetの辞書を作成
        saisei_dic = defaultdict(list)
        #音素ラベル，開始・終了タイミングを辞書に追加
        for j in range(len(data_list)):
            label =  data_list[j][2]
            if label != "silB" and label != "silE":
                x1 = int(fs * float(data_list[j][0]))
                x2 = int(fs * float(data_list[j][1]))
                saisei_dic[label].append([x1, x2])
        #音素の波形画像を出力
        for j in saisei_dic:
            for k in range(len(saisei_dic[j])):
                fig = plt.figure()
                ax = fig.add_subplot(1, 1, 1)
                ax.plot(v[saisei_dic[j][k][0]:saisei_dic[j][k][1]])
                plt.savefig('Phoneme_file/' + j + '_' + str(i) + '_' + str(k) + '.png')


ModuleNotFoundError: No module named 'cis'

In [2]:
#母音の音素に対してフォルマントを求め，線形予測分析(LPC)を使ってフォルマントの包絡とそのピークを抽出

#使用するパッケージ
import cis
import numpy as np
import scipy.signal
from levinson_durbin import autocorr, LevinsonDurbin
from collections import defaultdict
import matplotlib.pyplot as plt
from scipy.fftpack import fft

#プリエンファシス(高域強調)
def preEmphasis(wave, p=0.97):
    # 係数 (1.0, -p) のFIRフィルタを作成
    return scipy.signal.lfilter([1.0, -p], 1, wave)

boin_list = ["a","i","u","e","o"]

#データの読み込み, 音素ごとに辞書化
for i in range(1,100,1):
    data_list = []
    open_file = "lab_and_text/sound-"+str(i).zfill(3)+".lab"
    filename = "wav/sound-"+str(i).zfill(3)
    v ,fs = cis.wavread(filename+".wav")
    with open(open_file,"r") as f:
        data = f.readline().split()
        while data:
            data_list.append(data)
            data = f.readline().split()
        saisei_dic = defaultdict(list)
        for j in range(len(data_list)):
            label =  data_list[j][2]
            if label in boin_list:
                x1 = int(fs * float(data_list[j][0]))
                x2 = int(fs * float(data_list[j][1]))
                saisei_dic[label].append([x1, x2])
        for j in saisei_dic:
            for k in range(len(saisei_dic[j])):
                # LPC係数を求める(lpcの次数は要調整)
                lpcOrder = 16
                #短すぎるデータは扱いづらいので削除
                start = saisei_dic[j][k][0]
                end = saisei_dic[j][k][1]
                if (end - start) <= fs//200:
                    continue
                fig = plt.figure()
                ax = fig.add_subplot(1, 1, 1)
                #センター部分を使う
                center = (end + start) // 2
                cuttime = 0.04
                voice_data = v[center - int(cuttime / 2 * fs) : center + int(cuttime / 2 * fs)]
                #正規化
                voice_data = voice_data/max(abs(voice_data))
                #プリエンファシス
                p = 0.97
                voice_data = preEmphasis(voice_data, p)
                #ハミング窓
                hammingWindow = np.hamming(len(voice_data))
                voice_data = voice_data * hammingWindow    
                r = autocorr(voice_data, lpcOrder + 1)
                a, e  = LevinsonDurbin(r, lpcOrder)
                # LPC係数の振幅スペクトルを求める
                sample = len(voice_data)
                fscale = np.fft.fftfreq(sample, d = 1.0 / fs)[:sample//2]
                # オリジナル信号の対数スペクトル
                spec = np.abs(fft(voice_data, sample))
                logspec = 20 * np.log10(spec)
                ax.plot(fscale, logspec[:sample//2])
                # LPC対数スペクトル
                w, h = scipy.signal.freqz(np.sqrt(e), a, sample, "whole")
                lpcspec = np.abs(h)
                loglpcspec = 20 * np.log10(lpcspec)
                #出力をプロットしてみて出力
                ax.plot(fscale, loglpcspec[:sample//2], "r", linewidth=2)
                maxId = scipy.signal.argrelmax(loglpcspec[:sample//2],order=3)
                maxId = maxId[0]
                #とりあえず4つ分ぐらいのフォルマントの位置を出力
                ax.axvline(fscale[maxId[0]], ls = "--", color = "navy")
                ax.axvline(fscale[maxId[1]], ls = "--", color = "navy")
                ax.axvline(fscale[maxId[2]], ls = "--", color = "navy")
                ax.axvline(fscale[maxId[3]], ls = "--", color = "navy")
                plt.savefig('formant_file/'+j+'_'+str(i)+'_'+str(k)+'.png')

ModuleNotFoundError: No module named 'cis'